## Background

We want to be able to load R models using Python and describe, test and evaluate them using the ValidMind framework like we do with Python models. This notebook demonstrates how we can load R models either from an RDS file or by building the model in R directly in the notebook with the `rpy2` package. Either way, we can then use the ValidMind framework to run a TestPlan designed for the model (in this case, a simple ECM model).

In [ ]:
# lets import the required libraries
import os
import tempfile

import pandas as pd
import rpy2.robjects as robjects
from IPython.display import display_png
from PIL import Image as PILImage
from rpy2.robjects.packages import importr

# import the R packages
tidyverse = importr("tidyverse")
broom = importr("broom")
graphics = importr("graphics")
grdevices = importr("grDevices")

In [ ]:
# Load the RDS model we created earlier (in r-ecm-model notebook)
# alternatively, the model could be recreated in rpy2 from scratch
r_model = robjects.r["readRDS"]("r-ecm-model.rds")

In [ ]:
# lets run summary on the model
# in pure R, this would be: `summary(model)`
# for this, however we want to get a string representation of the summary
# so we can use it in python
summary = robjects.r["summary"](r_model)
summary_str = str(summary)
print(summary_str)

In [ ]:
# now lets something similar to run tidy, augment, and glance
# in pure R, this would be: `tidy(model)`, `augment(model)`, `glance(model)`
# however, we want to end up with a pandas dataframe containing the data in the Tibble created by these functions
tidy = robjects.r["tidy"](r_model)
tidy_df = pd.DataFrame(robjects.conversion.rpy2py(tidy))

augment = robjects.r["augment"](r_model)
augment_df = pd.DataFrame(robjects.conversion.rpy2py(augment))

glance = robjects.r["glance"](r_model)
glance_df = pd.DataFrame(robjects.conversion.rpy2py(glance))

# lets display the dataframes
display(tidy_df)
display(augment_df)
display(glance_df)

In [ ]:
# finally, lets plot the model and somehow get the plots into python
# in pure R, this would be: `plot(model)`
# for this, however we want to get a png image of the plots

# first of all, lets get a temporary file path that we can use to save the image
temp_file = tempfile.NamedTemporaryFile(suffix=".png")

# now lets save the image to the temporary file using grDevices package
grdevices.png(temp_file.name, width=1200, height=800)
graphics.par(mfrow=robjects.IntVector([2, 2]))
robjects.r["plot"](r_model) # creates 4 plots that will be combined into one image
grdevices.dev_off()

# now we split the image into the 4 plots
image = PILImage.open(temp_file.name)
width, height = image.size
plot_width = width / 2
plot_height = height / 2
plots = [
    image.crop((0, 0, plot_width, plot_height)),
    image.crop((plot_width, 0, width, plot_height)),
    image.crop((0, plot_height, plot_width, height)),
    image.crop((plot_width, plot_height, width, height))
]

# display the plots
for plot in plots:
    display_png(plot)

# and finally, lets delete the temporary file
os.remove(temp_file.name)